<a href="https://colab.research.google.com/github/fernandopaz1/Ciencia-de-Datos/blob/main/datosCovid_FrontendCafe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importamos pandas para manejo de datos, la libreria numerica numpy y la libreria para graficar matplotlib.pyplot 
<br> 

La siguiente linea configura la forma en que se muestran los floats en pandas

```
pd.options.display.float_format = '{:.2f}'.format 

```



In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.2f}'.format

Con en estas lineas montamos el google drive como si fuera un disco externo

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Tenemos que dar la direccion del cvs que quermos cargar relativo a nuestro drive \\
Si otra persona quiere usar este mismo codigo debe subir el cvs a una ruta similar en su cuenta de google drive o en la ruta que desee y cambiar el valor de filename \\
Con la función de pandas read_csv lee el contenido y nos devuelve un dataframe

In [59]:
filename = '/content/drive/My Drive/DatosFrontendCafe/casos_covid19.csv' 
datos = pd.read_csv(filename, dtype={})  # leyendo datos


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10,11,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Al leer los datos me dice que las conlumnas 10, 11 y 13 tienen tipos de datos mixtos, sugiere que activemos la lectura con baja memoria para ahorrar recursos \\

Abajo vemos opciones para visualizar esas columnas. Cuando ejecutamos solo muestra el resultado de la ultima, asi que conviene ejecutarlos de a uno o ponerlos dentro de un print() si queremos ver todos los resultados al mismo tiempo.

In [60]:
# datos.columns[10:14]
# datos.fallecido.unique()
# datos.comuna.value_counts()   # cuenta cuantas filas hay de cada tipo
type(datos.comuna[0])


numpy.float64

Pasamos las columnas de comuna y edad a entero, ya que inicialmente aparecian como float64

In [61]:
# pd.to_numeric(datos.comuna, downcast='integer', errors='ignore')
datos.comuna = datos.comuna.astype('Int64')
datos.edad = datos.edad.astype('Int64')

In [62]:
datos.edad

0          49
1          28
2          70
3          41
4          64
           ..
2528922    53
2528923    30
2528924    73
2528925    40
2528926    29
Name: edad, Length: 2528927, dtype: Int64

In [63]:
print(f'Columnas: {list(datos.columns)}')
print(f'Tamaño del dataframe {datos.shape}')

Columnas: ['numero_de_caso', 'fecha_apertura_snvs', 'fecha_toma_muestra', 'fecha_clasificacion', 'provincia', 'barrio', 'comuna', 'genero', 'edad', 'clasificacion', 'fecha_fallecimiento', 'fallecido', 'fecha_alta', 'tipo_contagio']
Tamaño del dataframe (2528927, 14)


In [64]:
datos.loc[0:2]  # Listando los primeros 3 elementos

,numero_de_caso,fecha_apertura_snvs,fecha_toma_muestra,fecha_clasificacion,provincia,barrio,comuna,genero,edad,clasificacion,fecha_fallecimiento,fallecido,fecha_alta,tipo_contagio
0,2775791,21SEP2020:00:00:00.000000,21SEP2020:00:00:00.000000,22SEP2020:00:00:00.000000,CABA,BARRACAS,4,masculino,49,confirmado,NaN,NaN,29SEP2020:00:00:00.000000,En Investigación
1,2775982,21SEP2020:00:00:00.000000,21SEP2020:00:00:00.000000,21SEP2020:00:00:00.000000,CABA,FLORES,7,femenino,28,confirmado,NaN,NaN,29SEP2020:00:00:00.000000,Comunitario
2,2779533,21SEP2020:00:00:00.000000,21SEP2020:00:00:00.000000,21SEP2020:00:00:00.000000,Buenos Aires,NaN,<NA>,femenino,70,confirmado,NaN,NaN,29SEP2020:00:00:00.000000,Comunitario


In [65]:
type(datos.fecha_apertura_snvs[0])

str

Como las fechas estan en formato string debemos pasarlas o otro formato (tipo datetime), lo cual nos permite hacer operaciones mas complejas como compararlos, ordenarlos, etc. <br>

Para esto hay que indicar que formato tiene el string para poder extraer dia, mes, año, etc. En este caso el formato es 

```
'%d%b%Y:%H:%M:%S.%f'
```



In [66]:
datos.fecha_apertura_snvs = pd.to_datetime(datos.fecha_apertura_snvs, format='%d%b%Y:%H:%M:%S.%f') # Muy importante pegarle a todo el formato de la fecha
datos.fecha_toma_muestra = pd.to_datetime(datos.fecha_toma_muestra, format='%d%b%Y:%H:%M:%S.%f')   # Ver si se puede hacer en menos lineas
datos.fecha_alta = pd.to_datetime(datos.fecha_alta, format='%d%b%Y:%H:%M:%S.%f')
datos.fecha_clasificacion = pd.to_datetime(datos.fecha_clasificacion, format='%d%b%Y:%H:%M:%S.%f')
datos.fecha_fallecimiento = pd.to_datetime(datos.fecha_fallecimiento, format='%d%b%Y:%H:%M:%S.%f')


Como habiamos visto datos.fecha_toma_muesta.value_counts() cuenta cuantas filas hay con la misma fecha de toma de muestra, es decir cuantos muestreos se hicieron en cada fecha. \\

Agregandole .tail() nos muestra la cola de ese resultado, es decir las fechas con menos muestras. tail(70) muestra las ultimas 70. \\

Algunos pueden ser directamente errores de tipeo o simplemente los primeros dias en que se empezo a testear, por lo que una estrategia sería descartar los todos las tomas de muestra previas a una fecha. Esta fecha puede ser la fecha en la que se empezaron a hacer testeos masivos.

In [67]:
datos.fecha_toma_muestra.value_counts().tail(70)

2020-02-25    13
2020-03-08    12
2020-02-28    11
2020-01-27    10
2020-03-04     9
              ..
2020-02-15     1
2020-02-29     1
2020-02-13     1
2020-01-25     1
2020-02-23     1
Name: fecha_toma_muestra, Length: 70, dtype: int64

Con las siguientes lineas vemos que hay valores de toma de muestra previos a la llegada del covid al pais por lo tanto habria que descartarlos

In [68]:
# Para ver en que rango cubre este dataset
print(datos['fecha_toma_muestra'].min())   # guarda 1974?  posible error de tipeo Outlier
print(datos['fecha_toma_muestra'].max())

# Boxplot 

# Probando si  compara fech
datos.fecha_toma_muestra.describe()


# datos.boxplot(column='fecha_toma_muestra', return_type='axes');
# plt.show()

1974-11-09 00:00:00
2021-05-27 00:00:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  


count                 2523273
unique                    520
top       2021-05-20 00:00:00
freq                    21714
first     1974-11-09 00:00:00
last      2021-05-27 00:00:00
Name: fecha_toma_muestra, dtype: object

Lo mismo aca  hay que descartar las fechas que no coincidan. Buscando en noticias encontramos que el primer fallecido en caba fue le 07/03/2020 [fuente](https://www.infobae.com/sociedad/2020/04/14/primera-muerte-por-coronavirus-en-argentina-era-un-hombre-que-habia-viajado-a-francia-y-estaba-internado-en-el-argerich/)

In [69]:
# datos.fecha_fallecimiento[0]   # los NaN los transforma a NaT
print(datos.fecha_fallecimiento.value_counts().tail(70))
datos.fecha_fallecimiento.describe()


2021-01-14    13
2020-04-27    13
2020-05-23    13
2020-11-25    13
2020-05-15    13
              ..
2020-03-09     1
2020-03-10     1
2020-03-18     1
2020-03-20     1
2020-03-23     1
Name: fecha_fallecimiento, Length: 70, dtype: int64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  This is separate from the ipykernel package so we can avoid doing imports until


count                   14743
unique                    440
top       2021-04-27 00:00:00
freq                       93
first     2020-03-07 00:00:00
last      2021-05-27 00:00:00
Name: fecha_fallecimiento, dtype: object

Vemos si hay mas de un fallecido el dia del primer desceso y cual fue la fecha de toma de muestra. \\

Hay 70 personas a las que se les hizo el test en una fecha posterior y se las registro como fallecidas el mismo dia del primer desceso

In [70]:
datos.fecha_toma_muestra[datos.fecha_fallecimiento == '2020-03-07']


1247994   2020-03-05
Name: fecha_toma_muestra, dtype: datetime64[ns]

Posible criterio de limpiado: tirar las filas donde el test se hizo luego del fallecimiento. Contra: descartariamos los casos de personas que fallecieron y luego se testearon



In [71]:
# datos = datos[datos.fecha_toma_muestra < datos.fecha_fallecimiento]  
datos = datos[datos.fecha_toma_muestra < datos.fecha_clasificacion]   # no se puede clasificar sin testear 

Una estrategia podria ser deshacernos de edades negativas y de edades muy altas (máximo 105). \\

Hay que ver tambien la edad de 0 años que tiene 1079 casos si se puede atribuir a error de tipeo u omisión del campo edad \\

In [72]:
print(f' Porcentaje de los que tienen edad de mas de 100 {datos.edad.value_counts().tail(30).sum()/len(datos.edad)}') # ver criterio de edad para tirar
print(datos.edad.value_counts().tail(20))
datos = datos[datos.edad >= 0]
datos = datos[datos.edad <= 105]


 Porcentaje de los que tienen edad de mas de 100 0.029342529263583973
 94     733
 95     577
 96     449
 97     317
 98     271
 99     167
 100    116
 101     59
 102     36
 103     25
 121     18
 105     11
 104     10
 221      2
 106      2
 108      2
 109      1
 131      1
 107      1
-10       1
Name: edad, dtype: Int64


Una vez filtradas las edades podemos normalizaras 

In [73]:
datos = datos.edad/datos.edad.max()

El criterio para descartar por toma de muestra puede ser la fecha del primer caso en argentina 2020-03-03 [fuente](https://www.argentina.gob.ar/noticias/salud-confirma-el-primer-caso-de-coronavirus-en-el-pais) o una fecha en la que comenzaron a hacer testeos masivos

In [76]:
datos = datos[datos.fecha_toma_muestra > "2020-03-03"]

AttributeError: ignored

In [ ]:
datos.edad.describe()

In [ ]:
datos.edad[datos]

In [ ]:
print(f'Los tipos de contagio son {datos.tipo_contagio.unique()}')
datos.groupby(['tipo_contagio', 'provincia']).count()  # duda

In [ ]:
print(f'Los genero {datos.genero.unique()}')
datos.groupby('genero').fallecido.count() 
# agrupo el dataframe por los que tienen mismo genero
# me fijo en la columna de fallecidos
 # nunique cuenta solo los que no son unicos

In [ ]:
datos.comuna.max()

In [ ]:


df_fallecidos = df.dropna(subset=['provincia','fallecido'])

print(df_fallecidos)
df_caba_fallecidos = df_fallecidos.filter(items= ['provincia','comuna', 'fallecido'])
df_caba_fallecidos = df_caba_fallecidos.loc[df['provincia'] == 'CABA']
print(df_caba_fallecidos)
histograma = df_caba_fallecidos.hist(bins=50)
plt.grid()
plt.show()